In [49]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

from tqdm import tqdm

Ниже будет предобработка текста и прогон генсима. По сути я просто взяла это из семинара по генсиму :) Еще я буду использовать модуль TQDM для измерения времени, тк в этой домашке все работает жутко долго.

In [2]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')

In [3]:
data = df.content.values.tolist()
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
data = [re.sub('\s+', ' ', sent) for sent in data]
data = [re.sub("\'", "", sent) for sent in data]

In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  
data_words = list(sent_to_words(data))

In [5]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [6]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in tqdm(texts):
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [7]:
data_words_nostops = remove_stopwords(data_words)
data_words_bigrams = make_bigrams(data_words_nostops)
spacy.load('en_core_web_sm')
nlp = spacy.load("en_core_web_sm")
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

100%|██████████| 11314/11314 [07:02<00:00, 26.79it/s]

[['where', 'thing', 'car', 'nntp_poste', 'host', 'park', 'line', 'wonder', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'model', 'name', 'engine', 'spec', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


Тут добавляем столбец с лемматизированными текстами в наш ДатаФрейм.

In [8]:
df['data_lemmatized'] = data_lemmatized

In [9]:
id2word = corpora.Dictionary(data_lemmatized)
corpus = [id2word.doc2bow(text) for text in data_lemmatized]

In [10]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('body', 1),
  ('bring', 1),
  ('call', 1),
  ('car', 5),
  ('could', 1),
  ('day', 1),
  ('door', 2),
  ('early', 1),
  ('engine', 1),
  ('enlighten', 1),
  ('funky', 1),
  ('history', 1),
  ('host', 1),
  ('info', 1),
  ('know', 1),
  ('late', 1),
  ('lerxst', 1),
  ('line', 1),
  ('look', 2),
  ('mail', 1),
  ('make', 1),
  ('model', 1),
  ('name', 1),
  ('neighborhood', 1),
  ('nntp_poste', 1),
  ('park', 1),
  ('production', 1),
  ('really', 1),
  ('rest', 1),
  ('see', 1),
  ('separate', 1),
  ('small', 1),
  ('spec', 1),
  ('sport', 1),
  ('thank', 1),
  ('thing', 1),
  ('where', 1),
  ('wonder', 1),
  ('year', 1)]]

Обучаем модель с рандомным кол-вом топиков просто чтобы посмотреть, верно ли мы все сделали ранее.

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

Модель обучилась, все хорошо. Проверяем топики

In [13]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.115*"player" + 0.103*"disk" + 0.058*"format" + 0.054*"network" + '
  '0.044*"data" + 0.033*"electronic" + 0.030*"factory" + 0.029*"rsa" + '
  '0.024*"datum" + 0.019*"regardless"'),
 (1,
  '0.030*"evidence" + 0.027*"believe" + 0.027*"say" + 0.026*"reason" + '
  '0.020*"claim" + 0.018*"people" + 0.017*"sense" + 0.015*"exist" + '
  '0.014*"may" + 0.014*"faith"'),
 (2,
  '0.082*"space" + 0.049*"image" + 0.029*"com" + 0.028*"orbit" + '
  '0.023*"launch" + 0.019*"mission" + 0.019*"earth" + 0.018*"plane" + '
  '0.017*"surface" + 0.015*"shuttle"'),
 (3,
  '0.157*"car" + 0.053*"bike" + 0.044*"engine" + 0.040*"ride" + 0.034*"ground" '
  '+ 0.027*"rider" + 0.027*"vehicle" + 0.024*"gas" + 0.024*"door" + '
  '0.022*"mile"'),
 (4,
  '0.701*"ax" + 0.020*"monitor" + 0.019*"character" + 0.017*"master" + '
  '0.017*"normal" + 0.013*"sorry" + 0.010*"font" + 0.009*"excuse" + '
  '0.009*"benefit" + 0.009*"sc"'),
 (5,
  '0.097*"law" + 0.089*"encryption" + 0.052*"security" + 0.051*"clipper" + '
  '

Маллет у меня отказался работать. Джава устанавливаться не хочет.

In [ ]:
mallet_path = '/Users/apple/Downloads/mallet-2.0.8/bin/mallet'
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=i, id2word=id2word)

Создаем функцию, выбирающую наилучшее количество топиков для имеющихся у нас данных. Я ставлю значения от 5 до 36 и шаг 5, иначе программа будет работать невозможно долго.

In [11]:
def choose_the_best(text, dicti, corpus):
    b = 0
    for i in tqdm(range(5,36,5)):
        lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dicti,
                                           num_topics=i, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
        coherence_model_lda = CoherenceModel(model=lda_model, texts=text, dictionary=dicti, coherence='c_v')
        coherence_lda = coherence_model_lda.get_coherence()
        print(coherence_lda)
        if coherence_lda > b:
            b = coherence_lda
            best = i
    return best

Прогоняем функцию и смотрим лучший вариант. Я распечатываю coherence каждый раз, чтоб убедиться, что функция работает корректно и действительно выбирает наилучший вариант.

In [24]:
best = choose_the_best(data_lemmatized, id2word, corpus)
print(best)

 14%|█▍        | 1/7 [01:25<08:35, 85.94s/it]

0.5269441951105925


 29%|██▊       | 2/7 [08:30<15:37, 187.56s/it]

0.5466767470131106


 43%|████▎     | 3/7 [12:59<14:07, 211.84s/it]

...

0.45480934007509133


 86%|████████▌ | 6/7 [24:16<03:46, 226.17s/it]

0.5036156445576092


100%|██████████| 7/7 [29:04<00:00, 249.20s/it]

0.474271454959223
10


Получили наилучший вариант 10. Далее работаем с ним. Сначала обучаем модель с нужным нам количеством топиков best.

In [12]:
best = 10

In [13]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=best, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

На всякий случай смотрим топики.

In [13]:
lda_model.show_topics(formatted=False, num_topics=best)

[(0,
  [('file', 0.027850587),
   ('window', 0.021941382),
   ('program', 0.02191324),
   ('use', 0.015722554),
   ('problem', 0.014346607),
   ('set', 0.013252458),
   ('run', 0.0122761605),
   ('entry', 0.011657789),
   ('image', 0.011535945),
   ('memory', 0.01123172)]),
 (1,
  [('people', 0.01993293),
   ('would', 0.015832035),
   ('say', 0.015714714),
   ('believe', 0.011121675),
   ('think', 0.011040432),
   ('reason', 0.011005266),
   ('make', 0.009948168),
   ('evidence', 0.009848934),
   ('may', 0.009772895),
   ('write', 0.008406777)]),
 (2,
  [('people', 0.0136572225),
   ('say', 0.0127685545),
   ('kill', 0.011285022),
   ('child', 0.010960716),
   ('government', 0.008870921),
   ('death', 0.0077233817),
   ('attack', 0.0073048105),
   ('day', 0.006969694),
   ('man', 0.0062017706),
   ('country', 0.0061769187)]),
 (3,
  [('line', 0.034186095),
   ('write', 0.02520564),
   ('would', 0.018932294),
   ('be', 0.016619122),
   ('go', 0.015758323),
   ('get', 0.0149134975),
   (

Создаем словарь словарей, где каждому ключу-топику соответствует значение-словарь, в котором ключ -- слово, а значение -- его вес в этом топике.

In [46]:
topic_dict = {}
for i in lda_model.show_topics(formatted=False, num_topics=31):
    word_weight = {}
    for word in i[1]:
        word_weight[word[0]] = word[1]
    topic_dict[i[0]] = word_weight

Создаем функцию, достающую топики из текста. Если ни одного слова не встретилось, присваиваем топик unknown.

In [47]:
def get_topic(text):
    dict_prob = {}
    for word in text:
        for i in range(len(topic_dict)):
            if word in topic_dict[i]:
                if i in dict_prob:
                    dict_prob[i] += topic_dict[i][word]
                else:
                    dict_prob[i] = topic_dict[i][word]
    if dict_prob != {}:
        topic = sorted(dict_prob.items(), key=lambda kv: kv[1], reverse=True)[0][0]
    else:
        topic = 'unknown'
    return topic

Смотрим, работает ли она на коротком простом тексте(он содержит два слова из топика 0).

In [48]:
print(get_topic(['program', 'run']))

0


Применяем ее к целому столбцу нашего датафрейма.

In [17]:
df['topic'] = df['data_lemmatized'].apply(get_topic)

Смотрим, корректно ли все получилось.

In [18]:
print(df)

                                                 content  target  \
0      From: lerxst@wam.umd.edu (where's my thing)\nS...       7   
1      From: guykuo@carson.u.washington.edu (Guy Kuo)...       4   
2      From: twillis@ec.ecn.purdue.edu (Thomas E Will...       4   
3      From: jgreen@amber (Joe Green)\nSubject: Re: W...       1   
4      From: jcm@head-cfa.harvard.edu (Jonathan McDow...      14   
...                                                  ...     ...   
11309  From: jim.zisfein@factory.com (Jim Zisfein) \n...      13   
11310  From: ebodin@pearl.tufts.edu\nSubject: Screen ...       4   
11311  From: westes@netcom.com (Will Estes)\nSubject:...       3   
11312  From: steve@hcrlgw (Steven Collins)\nSubject: ...       1   
11313  From: gunning@cco.caltech.edu (Kevin J. Gunnin...       8   

                   target_names  \
0                     rec.autos   
1         comp.sys.mac.hardware   
2         comp.sys.mac.hardware   
3                 comp.graphics   
4       

Далее решаем задание с tf-idf

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

У меня получилось чудовищно много текстов четвертого топика, поэтому на них падает и мой комп, и коллаб. 

In [20]:
df.groupby('topic').count()

,content,target,target_names,data_lemmatized
topic,,,,
0,513,513,513,513
1,702,702,702,702
2,41,41,41,41
3,8272,8272,8272,8272
4,163,163,163,163
5,154,154,154,154
6,372,372,372,372
7,1018,1018,1018,1018
8,45,45,45,45


Я разделю ДФ на 5 частей, обработаю каждую отдельно, затем соединю.

In [23]:
df1 = df[:2500].copy()

In [24]:
df2 = df[2500:5000].copy()

In [25]:
df3 = df[5000:7500].copy()

In [26]:
df4 = df[7500:10000].copy()

In [28]:
df5 = df[10000:].copy()

Создам функцию для вычисления tf-idf, которая будет запускать цикл для каждого топика. К текстам неопознанного топика я осознанно ее не применяю.

In [32]:
def insert_tfidf(df):
    for i in tqdm(range(best)):
        topic = df.loc[df['topic'] == i, 'data_lemmatized']
        topic_text = [' '.join(i) for i in topic]
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform(topic_text)
        feature_names = vectorizer.get_feature_names()
        dense = vectors.todense()
        denselist = dense.tolist()
        df_tfidf = pd.DataFrame(denselist, columns=feature_names).T
        df_tfidf.columns = list(df.index[df['topic'] == i])
        for col in df_tfidf.columns:
            top5 = list(df_tfidf[col].sort_values(ascending=False).head().index)
            df.loc[col, 'top_words'] = ' '.join(top5)

Применю функцию последовательно к каждому из 5 кусочков исходного датафрейма.

In [41]:
insert_tfidf(df1)
insert_tfidf(df2)
insert_tfidf(df3)
insert_tfidf(df4)
insert_tfidf(df5)

100%|██████████| 10/10 [00:15<00:00,  1.54s/it]


Склею получившиеся датафреймы в один.

In [38]:
dfs = [df1, df2, df3, df4, df5]

In [42]:
result = pd.concat(dfs)

Распечатаю получившийся датафрейм, чтобы показать, что все получилось правильно.

In [40]:
print(result)

                                                 content  target  \
0      From: lerxst@wam.umd.edu (where's my thing)\nS...       7   
1      From: guykuo@carson.u.washington.edu (Guy Kuo)...       4   
2      From: twillis@ec.ecn.purdue.edu (Thomas E Will...       4   
3      From: jgreen@amber (Joe Green)\nSubject: Re: W...       1   
4      From: jcm@head-cfa.harvard.edu (Jonathan McDow...      14   
...                                                  ...     ...   
11309  From: jim.zisfein@factory.com (Jim Zisfein) \n...      13   
11310  From: ebodin@pearl.tufts.edu\nSubject: Screen ...       4   
11311  From: westes@netcom.com (Will Estes)\nSubject:...       3   
11312  From: steve@hcrlgw (Steven Collins)\nSubject: ...       1   
11313  From: gunning@cco.caltech.edu (Kevin J. Gunnin...       8   

                   target_names  \
0                     rec.autos   
1         comp.sys.mac.hardware   
2         comp.sys.mac.hardware   
3                 comp.graphics   
4       

Уберу ненужные в итоге столбцы target и target_names (в задании нигде не сказано, что они требуются в финальном df)

In [44]:
del result['target']

In [43]:
del result['target_names']

Покажу вам самый финальный датафрейм.

In [45]:
result

,content,data_lemmatized,topic,top_words
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,"[where, thing, car, nntp_poste, host, park, li...",3,car door lerxst funky where
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,"[si, poll, final, summary, final, call, si, cl...",7,clock poll si final upgrade
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,"[question, engineering, computer, network, dis...",3,display machine bunch hear disk
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,"[division, line, host, amber, write, write, ar...",3,division chip quadrilateral weitek amber
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,"[question, organization, smithsonian_astrophys...",3,error bug warn memory expect
...,...,...,...,...
11309,From: jim.zisfein@factory.com (Jim Zisfein) \n...,"[migraine, city, ny_bis, reply, line, cheap, a...",3,patient eeg dn tumor brain
11310,From: ebodin@pearl.tufts.edu\nSubject: Screen ...,"[problem, screen, blank, sometimes, minor, phy...",7,blank wire screen sometimes computer
11311,From: westes@netcom.com (Will Estes)\nSubject:...,"[este, mount, case, organization, mail, group,...",4,cpu mount cooler este weight
11312,From: steve@hcrlgw (Steven Collins)\nSubject: ...,"[line, nntp_poste, host, article, write, boy, ...",5,point sphere see boy mercy


По сути coherence (связность) score показывает, насколько получившийся топик хорош в терминах интерпретируемости человеком (то есть насколько он вообще имеет смысл). Эта мера помогает нам не принимать во внимание топики, которые получились из-за простого статистического совпадения и не имеют практического смысла для решаемой задачи (непонятны человеку). Coherence score оценивается через измерение семантического сходства между наиболее важными для топика словами, то есть словами, получившими наибольший вес в процессе выделения топика. То есть мы смотрим, насколько часто наиболее важные для модели слова встречаются в текстах корпуса вместе, насколько они вообще в целом имеют связь между собой.